In [1]:
import spacy
import json
import typer
from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
import random
from wasabi import Printer
from pathlib import Path

from rel_pipeline import get_tokens, calculate_tensor, create_pairs

In [6]:
    vocab = Vocab()
    spacy.prefer_gpu()

    nlp = spacy.load("../../ner_component/training/model-best")
    mask_entities = ["CONDITION", "BENEFIT"]
    relations = ["RELATED"]

    dep = "../assets/dependencies.json"
    pos = "../assets/partofspeech.json"
    
    dep_list = None
    pos_list = None

    with open(dep, "r") as f:
        dep_list = json.load(f)

    with open(pos, "r") as f:
        pos_list = json.load(f)

In [8]:
lines = []
with open("../assets/annotations.jsonl","r", encoding="utf8") as jsonfile:
        for line in jsonfile:
            lines.append(line)

In [26]:
for line in lines[:1]:
    example = json.loads(line)
    if example["answer"] == "accept":
        #print(line)
        #words = [t["text"] for t in example["tokens"]]
        #spaces = [t["ws"] for t in example["tokens"]]
        #doc = Doc(vocab, words=words, spaces=spaces)

        doc = nlp(example["text"])
        ents_list = []

        for span in example["spans"]:
            ents_list.append(
                Span(
                    doc,
                    span["token_start"],
                    span["token_end"]+1,
                    span["label"],
                    )
            )
        print(example["text"])
        doc.set_ents(ents_list)
        tokens = get_tokens(doc)
        pairs = calculate_tensor(
            create_pairs(tokens),
            mask_entities,
            relations,
            dep_list,
            pos_list,
        )
        print(tokens)
        for relation in example["relations"]:
            key1 = (
                relation["head_span"]["token_start"],
                relation["head_span"]["token_end"],
                relation["child_span"]["token_start"],
                relation["child_span"]["token_end"],
            )
            key2 = (
                relation["child_span"]["token_start"],
                relation["child_span"]["token_end"],
                relation["head_span"]["token_start"],
                relation["head_span"]["token_end"],
            )

            if key1 in pairs:
                pairs[key1]["relation"][relation["label"]] = 1.0
            elif key2 in pairs:
                pairs[key2]["relation"][relation["label"]] = 1.0
            else:
                print("KEY NOT FOUND!!!")
                
        for pair in pairs:
            if pairs[pair]["relation"]["RELATED"] == 1:
                print(pairs[pair])

Excellent!. Perfect for immunity, I recommend
[{'text': 'Excellent', 'tokens': [Excellent], 'label': 'None', 'start': 0, 'end': 0, 'sent': Excellent!., 'pos': ['ADJ'], 'tensor': [array([-0.29230583, -0.46580127, -0.21889003,  0.96666473, -1.1603425 ,
        0.02000374,  0.01436359, -0.89690685, -1.3414408 , -0.39840296,
       -0.42928958, -0.5059026 ,  0.7554814 , -1.718676  , -1.1246542 ,
       -1.1432289 , -0.2672432 , -0.00613379, -0.18668798,  0.253735  ,
        1.9233689 , -0.20335832, -0.36120257,  0.7184354 ,  0.8227369 ,
       -1.0308807 ,  0.5862803 ,  0.00693938,  1.1425211 , -0.37691674,
       -0.40332317,  0.3089337 ,  0.5622795 , -0.75856423, -0.58313787,
       -0.36174476,  0.3761074 , -0.82905173, -1.1988621 ,  0.7858651 ,
        0.7994683 , -0.35482413, -0.05392182, -1.1110538 ,  0.76350904,
       -0.08054698,  0.16756037,  0.0411694 , -0.6720859 ,  0.6747135 ,
       -0.841336  ,  0.346258  ,  1.5153885 , -0.8952223 ,  0.44476068,
        0.46674013, -0.110307